In [6]:
import pandas as pd

In [7]:
df = pd.read_csv('data/clean_data.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44564 entries, 0 to 44563
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Delivery_person_Age             44564 non-null  float64
 1   Delivery_person_Ratings         44564 non-null  float64
 2   Weather_conditions              44564 non-null  object 
 3   Road_traffic_density            44564 non-null  object 
 4   Vehicle_condition               44564 non-null  int64  
 5   Type_of_order                   44564 non-null  object 
 6   Type_of_vehicle                 44564 non-null  object 
 7   multiple_deliveries             44564 non-null  float64
 8   Festival                        44564 non-null  object 
 9   City                            44564 non-null  object 
 10  Time_taken (min)                44564 non-null  int64  
 11  Week_days                       44564 non-null  object 
 12  Time_Orderd_Hours               

In [9]:
X = df.drop('Time_taken (min)', axis=1)

In [10]:
['Time_taken (min)', 'Road_traffic_density', 'multiple_deliveries',
       'Delivery_person_Ratings', 'Distance_Resturant_to_Location',
       'Delivery_person_Age', 'Festival', 'Vehicle_condition',
       'Time_Order_picked_Hours', 'Time_Orderd_Hours', 'Type_of_vehicle',
       'City']

['Time_taken (min)',
 'Road_traffic_density',
 'multiple_deliveries',
 'Delivery_person_Ratings',
 'Distance_Resturant_to_Location',
 'Delivery_person_Age',
 'Festival',
 'Vehicle_condition',
 'Time_Order_picked_Hours',
 'Time_Orderd_Hours',
 'Type_of_vehicle',
 'City']

In [11]:
X

,Delivery_person_Age,Delivery_person_Ratings,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Week_days,Time_Orderd_Hours,Time_Orderd_Minutes,Time_Order_picked_Hours,Time_Order_picked_Minutes,Distance_Resturant_to_Location
0,36.0,4.2,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,Saturday,21,55,22,10,10.280582
1,21.0,4.7,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,Sunday,14,55,15,5,6.242319
2,23.0,4.7,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,Friday,17,30,17,40,13.787860
3,34.0,4.3,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,Sunday,9,20,9,30,2.930258
4,24.0,4.7,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,Monday,19,50,20,5,19.396618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44559,30.0,4.8,Windy,High,1,Meal,motorcycle,0.0,No,Metropolitian,Thursday,11,35,11,45,1.489846
44560,21.0,4.6,Windy,Jam,0,Buffet,motorcycle,1.0,No,Metropolitian,Wednesday,19,55,20,10,11.007735
44561,30.0,4.9,Cloudy,Low,1,Drinks,scooter,0.0,No,Metropolitian,Friday,23,50,0,0,4.657195
44562,20.0,4.7,Cloudy,High,0,Snack,motorcycle,1.0,No,Metropolitian,Monday,13,35,13,40,6.232393


In [12]:
y = df['Time_taken (min)']

In [13]:
y

0        46
1        23
2        21
3        20
4        41
         ..
44559    32
44560    36
44561    16
44562    26
44563    36
Name: Time_taken (min), Length: 44564, dtype: int64

In [14]:
cat_cols = X.select_dtypes(include='object').columns

In [15]:
num_cols = X.select_dtypes(exclude='object').columns

In [16]:
cat_cols

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'Week_days'],
      dtype='object')

In [34]:
num_cols

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Vehicle_condition',
       'multiple_deliveries', 'Time_Orderd_Hours', 'Time_Orderd_Minutes',
       'Time_Order_picked_Hours', 'Time_Order_picked_Minutes',
       'Distance_Resturant_to_Location'],
      dtype='object')

In [17]:
Weather_conditions_categories =['Fog', 'Stormy', 'Sandstorms', 'Windy', 'Cloudy', 'Sunny']
Road_traffic_density_categories=['Jam', 'High', 'Medium', 'Low']
Type_of_order_categories=['Snack', 'Meal', 'Drinks', 'Buffet']
Type_of_vehicle_categories=['motorcycle', 'scooter', 'electric_scooter']
Festival_categories = ['No', 'Yes']
City_categories = ['Metropolitian', 'Urban', 'Semi-Urban']
Week_days_categories = ['Saturday', 'Sunday', 'Friday', 'Monday', 'Tuesday', 'Wednesday','Thursday']

In [18]:
df['Week_days'].unique()

array(['Saturday', 'Sunday', 'Friday', 'Monday', 'Tuesday', 'Wednesday',
       'Thursday'], dtype=object)

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [20]:
num_pipeline= Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy='median')),
        ("scaler", StandardScaler())
    ]
)

In [21]:
cat_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder', OrdinalEncoder(categories=[Weather_conditions_categories,Road_traffic_density_categories,Type_of_order_categories,Type_of_vehicle_categories,Festival_categories,City_categories,Week_days_categories])),
        # ('ordinalencoder', OrdinalEncoder(categories=[Road_traffic_density_categories,Festival_categories])),
        ('scaler', StandardScaler())
    ]
)


In [22]:
preprocessor=ColumnTransformer([
    ('num_pipeline', num_pipeline,num_cols ),
    ('cat_pipeline', cat_pipeline, cat_cols)
])

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [25]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [26]:
X_train

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,num_pipeline__Time_Orderd_Hours,num_pipeline__Time_Orderd_Minutes,num_pipeline__Time_Order_picked_Hours,num_pipeline__Time_Order_picked_Minutes,num_pipeline__Distance_Resturant_to_Location,cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_order,cat_pipeline__Type_of_vehicle,cat_pipeline__Festival,cat_pipeline__City,cat_pipeline__Week_days
0,-1.680369,-2.055867,-1.224799,-1.320419,-1.944468,0.863762,-1.750603,1.434564,-1.191266,0.307326,1.106912,-0.440359,-0.771166,-0.139653,-0.534993,0.488238
1,1.130443,0.523320,0.001582,0.441078,-1.944468,-0.076005,-1.750603,0.559316,-1.459228,-1.448616,1.106912,-0.440359,0.789686,-0.139653,-0.534993,-1.018485
2,0.603416,0.200921,-1.224799,3.964072,0.747215,-0.076005,0.725629,0.851065,1.195349,-1.448616,-1.298996,-1.334496,-0.771166,7.160600,-0.534993,-0.516244
3,0.427740,1.168117,0.001582,0.441078,-0.495100,1.177017,-0.226768,-1.774677,-0.641809,-0.277988,0.304942,-1.334496,0.789686,-0.139653,1.798387,0.488238
4,1.657471,-0.121477,-1.224799,0.441078,0.954268,-0.076005,0.916108,0.559316,-0.904513,0.892641,1.106912,0.453777,-0.771166,-0.139653,1.798387,-0.014003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35646,0.954767,-1.088672,-1.224799,0.441078,0.333110,-1.329028,0.344670,-0.607681,0.734050,0.307326,-1.298996,0.453777,-0.771166,-0.139653,-0.534993,-1.018485
35647,-0.977666,0.845718,-1.224799,0.441078,-1.323310,-0.076005,-1.179165,0.559316,-1.459305,0.892641,-0.497027,0.453777,-0.771166,-0.139653,-0.534993,-0.516244
35648,-1.329018,0.845718,0.001582,-1.320419,-0.909205,0.237250,-0.798206,0.559316,-0.658158,1.477955,-0.497027,-0.440359,0.789686,-0.139653,1.798387,1.492720
35649,-0.274963,1.168117,1.227964,-1.320419,1.161320,0.863762,1.106587,1.434564,-0.921855,1.477955,1.106912,1.347914,-0.771166,-0.139653,1.798387,-0.516244


In [27]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [28]:
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [29]:
regression.score(X_test,y_test)

0.5448248509501465

In [30]:
X_train

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,num_pipeline__Time_Orderd_Hours,num_pipeline__Time_Orderd_Minutes,num_pipeline__Time_Order_picked_Hours,num_pipeline__Time_Order_picked_Minutes,num_pipeline__Distance_Resturant_to_Location,cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_order,cat_pipeline__Type_of_vehicle,cat_pipeline__Festival,cat_pipeline__City,cat_pipeline__Week_days
0,-1.680369,-2.055867,-1.224799,-1.320419,-1.944468,0.863762,-1.750603,1.434564,-1.191266,0.307326,1.106912,-0.440359,-0.771166,-0.139653,-0.534993,0.488238
1,1.130443,0.523320,0.001582,0.441078,-1.944468,-0.076005,-1.750603,0.559316,-1.459228,-1.448616,1.106912,-0.440359,0.789686,-0.139653,-0.534993,-1.018485
2,0.603416,0.200921,-1.224799,3.964072,0.747215,-0.076005,0.725629,0.851065,1.195349,-1.448616,-1.298996,-1.334496,-0.771166,7.160600,-0.534993,-0.516244
3,0.427740,1.168117,0.001582,0.441078,-0.495100,1.177017,-0.226768,-1.774677,-0.641809,-0.277988,0.304942,-1.334496,0.789686,-0.139653,1.798387,0.488238
4,1.657471,-0.121477,-1.224799,0.441078,0.954268,-0.076005,0.916108,0.559316,-0.904513,0.892641,1.106912,0.453777,-0.771166,-0.139653,1.798387,-0.014003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35646,0.954767,-1.088672,-1.224799,0.441078,0.333110,-1.329028,0.344670,-0.607681,0.734050,0.307326,-1.298996,0.453777,-0.771166,-0.139653,-0.534993,-1.018485
35647,-0.977666,0.845718,-1.224799,0.441078,-1.323310,-0.076005,-1.179165,0.559316,-1.459305,0.892641,-0.497027,0.453777,-0.771166,-0.139653,-0.534993,-0.516244
35648,-1.329018,0.845718,0.001582,-1.320419,-0.909205,0.237250,-0.798206,0.559316,-0.658158,1.477955,-0.497027,-0.440359,0.789686,-0.139653,1.798387,1.492720
35649,-0.274963,1.168117,1.227964,-1.320419,1.161320,0.863762,1.106587,1.434564,-0.921855,1.477955,1.106912,1.347914,-0.771166,-0.139653,1.798387,-0.516244


In [31]:
import numpy as np

In [32]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_squre = r2_score(true, predicted)

    return mae, rmse, r2_squre


In [33]:
models = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "ElasticNet": ElasticNet(),
}

train_model_list =[]
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    y_predict = model.predict(X_test)

    mae, rmse, r2_squre = evaluate_model(y_test, y_predict)

    print(list(models.keys())[i])

    model_list.append(list(models.keys())[i])

    print("model Training performance")
    print("RMSE: ", rmse)
    print("MAE: ", mae)
    print("R2_Square: ", r2_squre)

    r2_list.append(r2_squre)

    print("="*35)
    print("\n")

LinearRegression
model Training performance
RMSE:  6.332743612437554
MAE:  5.023575445212794
R2_Square:  0.5448248509501465


Lasso
model Training performance
RMSE:  6.8126393520217094
MAE:  5.439853522295566
R2_Square:  0.4732245464390512


Ridge
model Training performance
RMSE:  6.332739283891878
MAE:  5.023570008013863
R2_Square:  0.5448254731909541


ElasticNet
model Training performance
RMSE:  6.854824581061658
MAE:  5.4833920607326565
R2_Square:  0.4666805496141315




In [38]:
import os
script_path = os.path.abspath('clean_data.csv')

In [40]:
parent_dir = os.path.dirname(script_path)

In [41]:
parent_dir

'd:\\ProjectStructure\\st_assigment\\notebooks'